In [1]:
MAX_DURATION = 8.0 # 2 bars
NOTE_SEPARATOR = '!'
REST_VALUE = '@'

import pickle

notes = pickle.load(open('./raggen-notes.bin', 'rb'))

In [2]:
from keras.utils import np_utils
import numpy as np

TIMESTEP = 0.5 # 16th notes
SEQ_LEN = int(4 / TIMESTEP) # 8 per bar

num_unique_notes = len(set(notes))
print(f'Number of unique notes: {num_unique_notes}')

# all unique pitches (including rests)
pitch_names = sorted(set(i for i in notes))
# map pitches to integers
note_to_int = dict((note, num) for num, note in enumerate(pitch_names))

data_X = []
data_y = []
for i in range(0, len(notes) - SEQ_LEN, 1):
    seq_in = notes[i:i + SEQ_LEN]
    data_X.append([note_to_int[n] for n in seq_in])

    seq_out = notes[i + SEQ_LEN]
    data_y.append(note_to_int[seq_out])
#end

X = np.reshape(data_X, (len(data_X), SEQ_LEN, 1))
X = X / float(len(notes))
y = np_utils.to_categorical(data_y)

Using TensorFlow backend.


Number of unique notes: 5806


In [3]:
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense
import keras.backend as K

K.clear_session()

model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dropout(0.3))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 8, 256)            264192    
_________________________________________________________________
dropout_1 (Dropout)          (None, 8, 256)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 5806)              1492142   
Total params: 2,281,646
Trainable params: 2,281,646
Non-trainable params: 0
_________________________________________________________________


In [4]:
model_path = './raggen-model.hdf5'
model.load_weights(model_path)

In [7]:
# reverse mapping
int_to_note = dict((num, note) for num, note in enumerate(pitch_names))

# if true, only uses the best prediction available
best_pred_only = False
# if best_pred_only is false, how many predictions to randomly choose from
num_best_preds = 2

# random start point
start_idx = np.random.randint(0, len(data_X)-1)
v1_pattern = data_X[start_idx]
print(f'Starting pattern: {v1_pattern}')
v1_output = []
for idx in range(100 * SEQ_LEN):
#     print(f'Pattern {idx}: {v1_pattern}')
    prediction_input = np.reshape(v1_pattern, (1, len(v1_pattern), 1))
    prediction_input = prediction_input / float(len(notes))
    prediction = model.predict(prediction_input)
    
    if best_pred_only:
        pred_idx = np.argmax(prediction[0])
    else:
        top_5_idx = np.argpartition(prediction[0], -num_best_preds)[-num_best_preds:]
        pred_idx = top_5_idx[np.random.randint(0, len(top_5_idx))]
    
    result = int_to_note[pred_idx]
    v1_output.append(result)

#     print(f'\tPredicted index: {pred_idx}')

    v1_pattern.append(pred_idx)
    v1_pattern = v1_pattern[1:len(v1_pattern)]
#end

Starting pattern: [1238, 3959, 4939, 1288, 4005, 3782, 4856, 3472]


In [8]:
from music21 import stream, duration, key, meter, note, chord, instrument

def split_note_duration(pattern):
    n, d = pattern.split('$')
    if '/' in d:
        a, b = d.split('/')
        d = float(a) / float(b)
    else:
        d = float(d)
    #end
    return n, d
#end

def notes_array_to_midi(notes_array):
    offset = 0.0
    output_notes = []
    
    for pattern in notes_array:
#         print(pattern)
        # handle chords (i.e. multiple notes split by NOTE_SEPARATOR)
        if NOTE_SEPARATOR in pattern:
            # adding all notes in the chord separately instead of in a chord.Chord.
            # this is because while music21 should support different length notes in a Chord,
            # it doesn't appear to work (uses the Chord's duration).
            for chord_note in pattern.split(NOTE_SEPARATOR):
                note_name, note_duration = split_note_duration(chord_note)
                new_note = note.Note(note_name)
                new_note.offset = offset
                new_note.storedInstrument = instrument.Piano
                new_note.duration = duration.Duration(note_duration)
                output_notes.append(new_note)
            #end
        #end
        else:
            note_name, note_duration = split_note_duration(pattern)
            # handle rests
            if REST_VALUE == note_name:
                new_rest = note.Rest()
                new_rest.offset = offset
                new_rest.duration = duration.Duration(note_duration)
                output_notes.append(new_rest)
            else:
                new_note = note.Note(note_name)
                new_note.offset = offset
                new_note.duration = duration.Duration(note_duration)
                output_notes.append(new_note)
        #end
        
        offset += TIMESTEP
    #end
    
    midi_stream = stream.Stream(output_notes)
    midi_stream.timeSignature = meter.TimeSignature('2/4')
    midi_stream.keySignature = key.KeySignature(0)
    midi_stream.write('midi', fp='./output.mid')
    return output_notes
#end

s = notes_array_to_midi(v1_output)
print('Done!')

Done!
